In [162]:
# Dependencies
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import pandas as pd
from sqlalchemy import create_engine
# We also need to do a pip install geopy before running this code
from geopy.geocoders import Nominatim
import numpy as np

In [157]:
# Make a reference to the comic_books_expanded.csv file path
csv_path = "../Data/rideshare_kaggle.csv"
# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")

# Import the rideshare_kaggle.csv file as a DataFrame
rides_df = pd.read_csv(csv_path, encoding="utf-8")

# Check the special characters imported correctly
rides_df



,id,timestamp,hour,day,month,datetime,timezone,source,destination,cab_type,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,1.544953e+09,9,16,12,2018-12-16 09:30:07,America/New_York,Haymarket Square,North Station,Lyft,...,0.1276,1544979600,39.89,1545012000,43.68,1544968800,33.73,1545012000,38.07,1544958000
1,4bd23055-6827-41c6-b23b-3c491f24e74d,1.543284e+09,2,27,11,2018-11-27 02:00:23,America/New_York,Haymarket Square,North Station,Lyft,...,0.1300,1543251600,40.49,1543233600,47.30,1543251600,36.20,1543291200,43.92,1543251600
2,981a3613-77af-4620-a42a-0c0866077d1e,1.543367e+09,1,28,11,2018-11-28 01:00:22,America/New_York,Haymarket Square,North Station,Lyft,...,0.1064,1543338000,35.36,1543377600,47.55,1543320000,31.04,1543377600,44.12,1543320000
3,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,1.543554e+09,4,30,11,2018-11-30 04:53:02,America/New_York,Haymarket Square,North Station,Lyft,...,0.0000,1543507200,34.67,1543550400,45.03,1543510800,30.30,1543550400,38.53,1543510800
4,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,1.543463e+09,3,29,11,2018-11-29 03:49:20,America/New_York,Haymarket Square,North Station,Lyft,...,0.0001,1543420800,33.10,1543402800,42.18,1543420800,29.11,1543392000,35.75,1543420800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693066,616d3611-1820-450a-9845-a9ff304a4842,1.543708e+09,23,1,12,2018-12-01 23:53:05,America/New_York,West End,North End,Uber,...,0.0000,1543683600,31.42,1543658400,44.76,1543690800,27.77,1543658400,44.09,1543690800
693067,633a3fc3-1f86-4b9e-9d48-2b7132112341,1.543708e+09,23,1,12,2018-12-01 23:53:05,America/New_York,West End,North End,Uber,...,0.0000,1543683600,31.42,1543658400,44.76,1543690800,27.77,1543658400,44.09,1543690800
693068,64d451d0-639f-47a4-9b7c-6fd92fbd264f,1.543708e+09,23,1,12,2018-12-01 23:53:05,America/New_York,West End,North End,Uber,...,0.0000,1543683600,31.42,1543658400,44.76,1543690800,27.77,1543658400,44.09,1543690800
693069,727e5f07-a96b-4ad1-a2c7-9abc3ad55b4e,1.543708e+09,23,1,12,2018-12-01 23:53:05,America/New_York,West End,North End,Uber,...,0.0000,1543683600,31.42,1543658400,44.76,1543690800,27.77,1543658400,44.09,1543690800


In [158]:
final_dataframe = rides_df[["hour", "day", "month", "datetime", 
                       "source", "destination", "cab_type", "name",
                       "price", "distance", "latitude", "longitude", "temperature",
                       "short_summary", "precipIntensity", "humidity", "windSpeed", "visibility", "dewPoint",
                       "pressure", "moonPhase"]]
final_dataframe


,hour,day,month,datetime,source,destination,cab_type,name,price,distance,...,longitude,temperature,short_summary,precipIntensity,humidity,windSpeed,visibility,dewPoint,pressure,moonPhase
0,9,16,12,2018-12-16 09:30:07,Haymarket Square,North Station,Lyft,Shared,5.0,0.44,...,-71.0330,42.34,Mostly Cloudy,0.0000,0.68,8.66,10.000,32.70,1021.98,0.30
1,2,27,11,2018-11-27 02:00:23,Haymarket Square,North Station,Lyft,Lux,11.0,0.44,...,-71.0330,43.58,Rain,0.1299,0.94,11.98,4.786,41.83,1003.97,0.64
2,1,28,11,2018-11-28 01:00:22,Haymarket Square,North Station,Lyft,Lyft,7.0,0.44,...,-71.0330,38.33,Clear,0.0000,0.75,7.33,10.000,31.10,992.28,0.68
3,4,30,11,2018-11-30 04:53:02,Haymarket Square,North Station,Lyft,Lux Black XL,26.0,0.44,...,-71.0330,34.38,Clear,0.0000,0.73,5.28,10.000,26.64,1013.73,0.75
4,3,29,11,2018-11-29 03:49:20,Haymarket Square,North Station,Lyft,Lyft XL,9.0,0.44,...,-71.0330,37.44,Partly Cloudy,0.0000,0.70,9.14,10.000,28.61,998.36,0.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693066,23,1,12,2018-12-01 23:53:05,West End,North End,Uber,UberXL,13.0,1.00,...,-71.0643,37.05,Partly Cloudy,0.0000,0.74,2.34,9.785,29.65,1023.57,0.82
693067,23,1,12,2018-12-01 23:53:05,West End,North End,Uber,UberX,9.5,1.00,...,-71.0643,37.05,Partly Cloudy,0.0000,0.74,2.34,9.785,29.65,1023.57,0.82
693068,23,1,12,2018-12-01 23:53:05,West End,North End,Uber,Taxi,NaN,1.00,...,-71.0643,37.05,Partly Cloudy,0.0000,0.74,2.34,9.785,29.65,1023.57,0.82
693069,23,1,12,2018-12-01 23:53:05,West End,North End,Uber,Black SUV,27.0,1.00,...,-71.0643,37.05,Partly Cloudy,0.0000,0.74,2.34,9.785,29.65,1023.57,0.82


In [159]:
# Rename the headers to be more explanatory
final_dataframe = final_dataframe.rename(columns={"latitude": "destination_lat",
                                        "longitude": "destination_long"})
final_dataframe

,hour,day,month,datetime,source,destination,cab_type,name,price,distance,...,destination_long,temperature,short_summary,precipIntensity,humidity,windSpeed,visibility,dewPoint,pressure,moonPhase
0,9,16,12,2018-12-16 09:30:07,Haymarket Square,North Station,Lyft,Shared,5.0,0.44,...,-71.0330,42.34,Mostly Cloudy,0.0000,0.68,8.66,10.000,32.70,1021.98,0.30
1,2,27,11,2018-11-27 02:00:23,Haymarket Square,North Station,Lyft,Lux,11.0,0.44,...,-71.0330,43.58,Rain,0.1299,0.94,11.98,4.786,41.83,1003.97,0.64
2,1,28,11,2018-11-28 01:00:22,Haymarket Square,North Station,Lyft,Lyft,7.0,0.44,...,-71.0330,38.33,Clear,0.0000,0.75,7.33,10.000,31.10,992.28,0.68
3,4,30,11,2018-11-30 04:53:02,Haymarket Square,North Station,Lyft,Lux Black XL,26.0,0.44,...,-71.0330,34.38,Clear,0.0000,0.73,5.28,10.000,26.64,1013.73,0.75
4,3,29,11,2018-11-29 03:49:20,Haymarket Square,North Station,Lyft,Lyft XL,9.0,0.44,...,-71.0330,37.44,Partly Cloudy,0.0000,0.70,9.14,10.000,28.61,998.36,0.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693066,23,1,12,2018-12-01 23:53:05,West End,North End,Uber,UberXL,13.0,1.00,...,-71.0643,37.05,Partly Cloudy,0.0000,0.74,2.34,9.785,29.65,1023.57,0.82
693067,23,1,12,2018-12-01 23:53:05,West End,North End,Uber,UberX,9.5,1.00,...,-71.0643,37.05,Partly Cloudy,0.0000,0.74,2.34,9.785,29.65,1023.57,0.82
693068,23,1,12,2018-12-01 23:53:05,West End,North End,Uber,Taxi,NaN,1.00,...,-71.0643,37.05,Partly Cloudy,0.0000,0.74,2.34,9.785,29.65,1023.57,0.82
693069,23,1,12,2018-12-01 23:53:05,West End,North End,Uber,Black SUV,27.0,1.00,...,-71.0643,37.05,Partly Cloudy,0.0000,0.74,2.34,9.785,29.65,1023.57,0.82


In [160]:
#final_dataframe.insert(11, "source_lat", "", True)
#final_dataframe.insert(12, "source_lat", "", True)
#final_dataframe
#final_dataframe.source.unique()
final_dataframe.drop(final_dataframe[final_dataframe.source == 55.94567815].index, inplace=True)
final_dataframe.source.unique()

array(['Haymarket Square', 'Back Bay', 'North End', 'North Station',
       'Beacon Hill', 'Boston University', 'Fenway', 'South Station',
       'Theatre District', 'West End', 'Financial District',
       'Northeastern University'], dtype=object)

In [147]:
def get_lat(address):
    location = geolocator.geocode(address)
    return location

In [148]:
lat_long1 = get_lat('Haymarket Square, Boston, MA')
lat_long2 = get_lat('Back Bay, Boston, MA')
lat_long3 = get_lat('North End, Boston, MA')
lat_long4 = get_lat('North Station, Boston, MA')
lat_long5 = get_lat('Beacon Hill, Boston, MA')
lat_long6 = get_lat('Boston University, Boston, MA')
lat_long7 = get_lat('Fenway, Boston, MA')
lat_long8 = get_lat('South Station, Boston, MA')
lat_long9 = get_lat('Theatre District, Boston, MA')
lat_long10 = get_lat('West End, Boston, MA')
lat_long11 = get_lat('Financial District, Boston, MA')
lat_long12 = get_lat('Northeastern University, Boston, MA')

In [149]:
lat_long1.latitude

42.3629502

In [150]:
lat_long1.longitude

-71.0578447

In [163]:
conditions = [
    (final_dataframe['source'] == 'Haymarket Square'),
    (final_dataframe['source'] == 'Back Bay'),
    (final_dataframe['source'] == 'North End'),
    (final_dataframe['source'] == 'North Station'),
    (final_dataframe['source'] == 'Beacon Hill'),
    (final_dataframe['source'] == 'Boston University'),
    (final_dataframe['source'] == 'Fenway'),
    (final_dataframe['source'] == 'South Station'),
    (final_dataframe['source'] == 'Theatre District'),
    (final_dataframe['source'] == 'West End'),
    (final_dataframe['source'] == 'Financial District'),
    (final_dataframe['source'] == 'Northeastern University')
    ]

# create a list of the values we want to assign for each condition
values = [lat_long1.latitude, lat_long2.latitude, lat_long3.latitude, lat_long4.latitude, lat_long5.latitude, \
         lat_long6.latitude, lat_long7.latitude, lat_long8.latitude, lat_long9.latitude, lat_long10.latitude, \
         lat_long11.latitude, lat_long12.latitude,]

# create a new column and use np.select to assign values to it using our lists as arguments
final_dataframe['source_lat'] = np.select(conditions, values)

# display updated DataFrame
final_dataframe.head()

,hour,day,month,datetime,source,destination,cab_type,name,price,distance,...,temperature,short_summary,precipIntensity,humidity,windSpeed,visibility,dewPoint,pressure,moonPhase,source_lat
0,9,16,12,2018-12-16 09:30:07,Haymarket Square,North Station,Lyft,Shared,5.0,0.44,...,42.34,Mostly Cloudy,0.0000,0.68,8.66,10.000,32.70,1021.98,0.30,42.36295
1,2,27,11,2018-11-27 02:00:23,Haymarket Square,North Station,Lyft,Lux,11.0,0.44,...,43.58,Rain,0.1299,0.94,11.98,4.786,41.83,1003.97,0.64,42.36295
2,1,28,11,2018-11-28 01:00:22,Haymarket Square,North Station,Lyft,Lyft,7.0,0.44,...,38.33,Clear,0.0000,0.75,7.33,10.000,31.10,992.28,0.68,42.36295
3,4,30,11,2018-11-30 04:53:02,Haymarket Square,North Station,Lyft,Lux Black XL,26.0,0.44,...,34.38,Clear,0.0000,0.73,5.28,10.000,26.64,1013.73,0.75,42.36295
4,3,29,11,2018-11-29 03:49:20,Haymarket Square,North Station,Lyft,Lyft XL,9.0,0.44,...,37.44,Partly Cloudy,0.0000,0.70,9.14,10.000,28.61,998.36,0.72,42.36295


In [165]:
values2 = [lat_long1.longitude, lat_long2.longitude, lat_long3.longitude, lat_long4.longitude, lat_long5.longitude, \
         lat_long6.longitude, lat_long7.longitude, lat_long8.longitude, lat_long9.longitude, lat_long10.longitude, \
         lat_long11.longitude, lat_long12.longitude,]

# create a new column and use np.select to assign values to it using our lists as arguments
final_dataframe['source_longitude'] = np.select(conditions, values2)


In [166]:
# display updated DataFrame
final_dataframe.head()

,hour,day,month,datetime,source,destination,cab_type,name,price,distance,...,short_summary,precipIntensity,humidity,windSpeed,visibility,dewPoint,pressure,moonPhase,source_lat,source_longitude
0,9,16,12,2018-12-16 09:30:07,Haymarket Square,North Station,Lyft,Shared,5.0,0.44,...,Mostly Cloudy,0.0000,0.68,8.66,10.000,32.70,1021.98,0.30,42.36295,-71.057845
1,2,27,11,2018-11-27 02:00:23,Haymarket Square,North Station,Lyft,Lux,11.0,0.44,...,Rain,0.1299,0.94,11.98,4.786,41.83,1003.97,0.64,42.36295,-71.057845
2,1,28,11,2018-11-28 01:00:22,Haymarket Square,North Station,Lyft,Lyft,7.0,0.44,...,Clear,0.0000,0.75,7.33,10.000,31.10,992.28,0.68,42.36295,-71.057845
3,4,30,11,2018-11-30 04:53:02,Haymarket Square,North Station,Lyft,Lux Black XL,26.0,0.44,...,Clear,0.0000,0.73,5.28,10.000,26.64,1013.73,0.75,42.36295,-71.057845
4,3,29,11,2018-11-29 03:49:20,Haymarket Square,North Station,Lyft,Lyft XL,9.0,0.44,...,Partly Cloudy,0.0000,0.70,9.14,10.000,28.61,998.36,0.72,42.36295,-71.057845


In [167]:
final_dataframe[(final_dataframe.source == 'Back Bay')]

,hour,day,month,datetime,source,destination,cab_type,name,price,distance,...,short_summary,precipIntensity,humidity,windSpeed,visibility,dewPoint,pressure,moonPhase,source_lat,source_longitude
6,5,26,11,2018-11-26 05:03:00,Back Bay,Northeastern University,Lyft,Lyft XL,10.5,1.08,...,Overcast,0.0000,0.91,0.53,4.675,39.54,1014.11,0.64,42.350549,-71.080311
7,19,2,12,2018-12-02 19:53:04,Back Bay,Northeastern University,Lyft,Lux Black,16.5,1.08,...,Light Rain,0.0246,0.93,3.38,3.052,48.02,1004.33,0.86,42.350549,-71.080311
8,6,3,12,2018-12-03 06:28:02,Back Bay,Northeastern University,Lyft,Shared,3.0,1.08,...,Foggy,0.0000,0.96,1.25,1.413,44.50,1001.06,0.89,42.350549,-71.080311
9,10,27,11,2018-11-27 10:45:22,Back Bay,Northeastern University,Lyft,Lux Black XL,27.5,1.08,...,Light Rain,0.0624,0.93,6.87,2.686,43.52,989.98,0.68,42.350549,-71.080311
10,16,30,11,2018-11-30 16:13:03,Back Bay,Northeastern University,Lyft,Lux,13.5,1.08,...,Clear,0.0000,0.62,3.46,9.920,27.99,1016.84,0.79,42.350549,-71.080311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693005,5,30,11,2018-11-30 05:23:02,Back Bay,North End,Uber,UberPool,14.5,7.36,...,Clear,0.0000,0.75,4.74,9.990,26.76,1013.87,0.79,42.350549,-71.080311
693006,18,13,12,2018-12-13 18:15:03,Back Bay,North End,Uber,Black,36.0,7.36,...,Mostly Cloudy,0.0000,0.53,2.47,9.926,17.58,1033.59,0.21,42.350549,-71.080311
693007,18,13,12,2018-12-13 18:15:03,Back Bay,North End,Uber,Black SUV,49.5,7.36,...,Mostly Cloudy,0.0000,0.53,2.47,9.926,17.58,1033.59,0.21,42.350549,-71.080311
693008,18,13,12,2018-12-13 18:15:03,Back Bay,North End,Uber,UberX,16.0,7.36,...,Mostly Cloudy,0.0000,0.53,2.47,9.926,17.58,1033.59,0.21,42.350549,-71.080311


In [168]:
final_df = pd.DataFrame(final_dataframe)

# Need to replace the name of the sqlite file
engine = create_engine('sqlite:///Uber_Lyft_data.db')
# Need to replace the table name
final_df.to_sql(name='UberLyft', con=engine, if_exists='append', index=False)


In [170]:
# Need to read from right table to verify insertion of data
pd.read_sql_query('select count(*) from UberLyft', con=engine).head()

,count(*)
0,693071
